In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import random

#sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set(rc={'figure.figsize':(6,10)})

In [ ]:
import sys
sys.setrecursionlimit(4000)   #2000
print(sys.getrecursionlimit())

In [ ]:
size = 1000
width= 20

# Demo

In [ ]:
img = cv2.imread("img4.jpg")[:size,:size,:]     # 4 best
plt.figure(figsize=(8,8))
plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

In [ ]:
# colored image
img = cv2.imread("img4.jpg")     # 4 best
img = img[:size,:size,:3]
print(img.shape)
x,y,z = img.shape
for i in range(0,y,width):
    cv2.line(img,(i,0),(i,y), (100,100,100),1)
for j in range(0,x,width):
    cv2.line(img,(0,j),(y,j), (100,100,100),1)
"""
while True:   
    cv2.imshow("window", img)
    if cv2.waitKey(0) == 27:
        break
cv2.destroyAllWindows()
"""

cv2.imwrite('grid2_1000.jpg',img) #to save mean image
temp = cv2.imread("grid2_1000.jpg")
plt.figure(figsize=(8,8))
plt.imshow(cv2.cvtColor(temp,cv2.COLOR_BGR2RGB))

In [ ]:
# Edit

In [ ]:
img = cv2.imread("img4.jpg")[:size,:size,:]
mean_img = img.copy()
img_transition_value = np.zeros((size//width,size//width))
for i in range(0,size,width):
    for j in range(0,size,width):
        # got all the points
        pixel_value = np.mean(img[i:i+width,j:j+width,:])
        if pixel_value >210:              #200 didn't work
            mean_img[i:i+width,j:j+width,:] = 255
        else:
            mean_img[i:i+width,j:j+width,:] = pixel_value
        #print(i,j)
cv2.imwrite('mean2_1000.jpg',mean_img) #to save mean image
temp = cv2.imread("mean2_1000.jpg")
plt.figure(figsize=(8,8))
plt.imshow(cv2.cvtColor(temp,cv2.COLOR_BGR2RGB))

In [ ]:
# kernel = np.ones((5,5), np.uint8)  # check how is better with or without dialation
# img_dilation = cv2.dilate(mean_img, kernel, iterations=1)
img_dilation = mean_img.copy()

In [ ]:
cv2.imwrite("dilated_5.jpg",img_dilation)

In [ ]:
img_dilation.shape

In [ ]:
sns.distplot(img_dilation)

In [ ]:
img_transition_value=np.where(img_dilation<230,0,255) # 230
cv2.imwrite("dilated_5_2.jpg",img_transition_value)

tmp = cv2.imread('dilated_5_2.jpg')
plt.figure(figsize=(8,8))
plt.imshow(cv2.cvtColor(temp,cv2.COLOR_BGR2RGB))

In [ ]:
img_transition_value=(img_transition_value[::width,::width,0]).copy()

In [ ]:
img_transition_value

In [ ]:
sns.distplot(img_transition_value)

In [ ]:
img_transition_value=np.where(img_transition_value<120,-100,-1) # 230

In [ ]:
img_transition_value

In [ ]:
sns.distplot(img_transition_value)

In [ ]:
plt.figure(figsize=(10,8))  # without dialation
sns.heatmap(img_transition_value)

In [ ]:
plt.figure(figsize=(10,8))  # without dialation seems better
sns.heatmap(img_transition_value)

# MAIN Function

In [ ]:
img = cv2.imread("img4.jpg")[:size,:size,:]    #[:1000,:1000,:]
def Calculate_transition_matrix(img):
    global width
    global size
    mean_img = img.copy()
    img_transition_value = np.zeros((size//width,size//width))
    for i in range(0,size,width):
        for j in range(0,size,width):
            # got all the points
            pixel_value = np.mean(img[i:i+width,j:j+width,:])
            if pixel_value >210:              #200 didn't work
                mean_img[i:i+width,j:j+width,:] = 250
            else:
                mean_img[i:i+width,j:j+width,:] = 0 #pixel_value
    #kernel = np.ones((5,5), np.uint8)
    #img_dilation = cv2.dilate(mean_img, kernel, iterations=1) 
    img_dilation = mean_img.copy()
    img_transition_value=np.where(img_dilation<230,0,255) # 230
    #cv2.imwrite("dilated_5_2.jpg",img_transition_value)  # final image look BGR looks black and white
    img_transition_value=img_transition_value[::width,::width,0]
    img_transition_value=np.where(img_transition_value<120,-10,-1) # 230 
    return img_transition_value

In [ ]:
def Sweep(Value, Policy,terminators,transition_value):
    # create transition matrix globally that is not possible since we will create dynamic wall
    New_Value = Value.copy()
    delta = 0
    x,y = Value.shape
    for i in range(x):  # x
        left = 0
        right = 0
        top = 0
        bottom = 0
        for j in range(y):  # y
            # (i,j)  
            if (i,j) in terminators:
                pass
            else:
                same= Value[i,j]
                if i==0:
                    left = same
                else:
                    left = Value[i-1,j]
                if i==x-1:
                    right= same
                else:
                    right = Value[i+1,j]
                if j==0:
                    top= same
                else:
                    top = Value[i,j-1]
                if j==y-1:
                    bottom= same
                else:
                    bottom = Value[i ,j+1]
#                 if (i,j) in red:
#                     transition_reward = -10
#                 else:
#                     transition_reward = -1
                transition_reward = transition_value[i,j]
                total_value = Policy[(y)*i+j,0]*(transition_reward+left) + Policy[(y)*i+j,1]*(transition_reward+right) + Policy[(y)*i+j,2]*(transition_reward+top) + Policy[(y)*i+j,3]*(transition_reward+bottom)
                #print(total_value, New_Value[i,j])
                delta = max(delta, np.abs(total_value- Value[i,j]))
                New_Value[i,j] = total_value    
    #print("Sweep")
    return New_Value, delta

In [ ]:
def Improve_Policy(Value, Policy):
    New_Policy = Policy.copy()
    #terminators = (0,1), (0,2),(0,3), (2,0),(2,1),(2,2)
    x,y = Value.shape
    for i in range(x):
        for j in range(y):  # y
            # (i,j)  
            left = 0
            right = 0
            top = 0
            bottom = 0
            same= Value[i,j]
            if i==0:
                left = same
            else:
                left = Value[i-1,j]
            if i==x-1:
                right= same
            else:
                right = Value[i+1,j]
            if j==0:
                top= same
            else:
                top = Value[i,j-1]
            if j==y-1:
                bottom= same
            else:
                bottom = Value[i ,j+1]
            my_list = [left,right, top,bottom]
            max_val = max(my_list)
            my_list = np.array(my_list)
            my_list = (my_list==max_val).astype(int)
            my_list = my_list/np.sum(my_list)
            New_Policy[(y)*i+j,:] = my_list
    #print("Improve_Policy")
    return New_Policy

In [ ]:
call =0
def Evaluate_value(Value, Policy,theta,terminators,transition_value):
    Policy = Policy.copy()
    theta = theta
    terminators = terminators.copy()
    Policy = Policy.copy()
    global call
    call += 1
    #terminators = (0,1), (0,2),(0,3), (2,0),(2,1),(2,2)
    delta = float(np.inf)
    while delta> theta:
        delta = 0
        Value, delta = Sweep(Value, Policy, terminators,transition_value)
        print(delta,theta, delta>theta)
    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    New_Policy = Improve_Policy(Value, Policy)
    if np.all(Policy==New_Policy):
        #print("Evaluate_value_")
        return Value, New_Policy, theta, terminators
    else:
        # repeat
        #print("Evaluate_value")
        return Evaluate_value(Value, New_Policy,theta, terminators,transition_value)

In [ ]:
def Calculate_optimal(Grid_shape,no_actions, terminators,img_mask):
    x,y = Grid_shape
    State_action = np.ones((x*y,no_actions))*(1/no_actions)  #######  L     R     T     B
    Value = np.zeros((x,y))
    theta = 0.001
    
    transition_value = Calculate_transition_matrix(img_mask)
    l,m,th,tr =Evaluate_value(Value,State_action,9.5, terminators, transition_value) #l:- value function m:- optimal action
    #print("Calculate_optimal")         # 80 for -100 , 9.5 for -10 , 100 for -1000  60 for -80
    return l,m              

In [ ]:
optimal_path = np.zeros((50,50))
collect_pts = []
def trace_path(optimal_value, optimal_policy,initial_pt):
    global optimal_path
    global width 
    global collect_pts
    x,y = initial_pt  # 48,25
    if (x,y) == HOME:
        # stop
        print("REACHED HOME")
        return
    if x<0 or x>= 50 or y<0 or y>= 50:
        # stop execution
        return
    # Plot
    a1 = width*y
    b1 = width*x
    a2 = width*y + width
    b2 = width*x + width
    
    #
    policy_index = x*50 + y
    l,r,t,b = optimal_policy[policy_index,:]
    max_index = (l,r,t,b).index(max(l,r,t,b)) # pick the first optimal index
    indexes = [i for i,j in enumerate((l,r,t,b)) if j==max(l,r,t,b)]
    max_index = random.choice(indexes)
    if max_index==0:
        # left
        #cv2.circle(img, ((a1+a2)//2,(b1+b2)//2),3,(0,250,0),2)
        #cv2.line(img,((a1+a2)//2,b1),((a1+a2)//2,b2), (0,0,0),3)
        trace_path(optimal_value, optimal_policy,(x-1,y))
        optimal_path[x-1:x,y:y+1] += 1
    elif max_index==1:
        # right
        #cv2.circle(img, ((a1+a2)//2,(b1+b2)//2),3,(0,250,0),2)
        #cv2.line(img,((a1+a2)//2,b1),((a1+a2)//2,b2), (0,0,0),3)
        trace_path(optimal_value, optimal_policy,(x,y+1))
        optimal_path[x:x+1,y+1:y+2] += 1
    elif max_index==2:
        # top
        #cv2.circle(img, ((a1+a2)//2,(b1+b2)//2),3,(0,250,0),2)
        #cv2.line(img,(a1,(b1+b2)//2),(a2,(b1+b2)//2), (0,0,0),3)
        trace_path(optimal_value, optimal_policy,(x,y-1))
        optimal_path[x:x+1,y-1:y] += 1
    elif max_index==3:
        # bottom 
        #cv2.circle(img, ((a1+a2)//2,(b1+b2)//2),3,(0,250,0),2)
        #cv2.line(img,(a1,(b1+b2)//2),(a2,(b1+b2)//2), (0,0,0),3)
        trace_path(optimal_value, optimal_policy,(x,y+1))
        optimal_path[x:x+1,y+1:y+2] += 1
    else:
        print("ERROR")

In [ ]:
def draw_path():
    global optimal_path
    global img
    global width
    global size
    print('calculating')
    Full_matrix = (np.repeat(np.repeat(optimal_path,width,axis=0),width,axis=1)*500).astype(int)
    img = ((np.clip((img - Full_matrix.reshape(size,size,1)).astype(int),0,255)).astype(int)).astype(np.uint8)

In [ ]:
A = np.array([[0,3],[5,0]])
A

In [ ]:
np.repeat(np.repeat(A,3,axis=0),3,axis=1)

In [ ]:
drag = False
HOME =0
AGENT = 0
Grid_shape = (50,50) #(100,100) # later set automatically
no_actions = 4
terminatos = []
optimal_path = np.zeros((50,50))
width = 10 #20
size =  500 #1000

# PATH
img_mask = cv2.imread("dilated_5_2.jpg")[:size,:size,:] #[:1000,:1000,:]  # img4.jpg
img = cv2.imread("img4.jpg")[:size,:size,:] #[:1000,:1000,:]  # img4.jpg
#img = cv2.cvtColor(img, cv2.Color_BGR2RGB)
cv2.namedWindow("window")
optimal_value = 0
state =0 
"""
state:click
0       No
1       set home
2       set car
3       start optimizing
4       start game
5       loop----in 5
"""



def draw(event, x, y, flag, param):
    global drag
    global state
    global HOME
    global AGENT
    global optimal_value
    global img
    global Grid_shape
    global no_action
    global terminators
    global optimal_path
    global width
    global size
    
    if event == cv2.EVENT_FLAG_LBUTTON:
        drag = True
    if event == cv2.EVENT_LBUTTONUP:
        drag=False
        #cv2.circle(img, (x,y),20,(0,0,0),3)
        if state == 0:
            # draw home
            HOME = (y//width,x//width)
            print("HOME location")
            print(HOME)
            cv2.rectangle(img, (x+width,y-width),(x-width,y+width),(0,0,250),3)
            state =1
        elif state == 1:
            # draw car
            AGENT = [y//width,x//width]
            print("AGENT location")
            print(AGENT)
            cv2.rectangle(img, (x+width,y-width),(x-width,y+width),(250,0,0),3)
            state = 2
        elif state == 2:
            state = 3
            # show color
            terminators = [HOME]
            print("terminating locations")
            print(terminators)
            
            img_mask = cv2.imread("img4.jpg")[:size,:size,:] #[:1000,:1000,:]  # img4.jpg # this is the fresh copy not affected by any thing 
            print()
            print("Calculating...")
            optimal_value,optimal_policy = Calculate_optimal(Grid_shape,no_actions, terminators,img_mask)
            print('OPTIMAL POLICY')
            print(optimal_value)
            l,m = optimal_value.shape  # (100,100)
            #print(img.shape)
            trace_path(optimal_value,optimal_policy,AGENT)
            #img[HOME[0]*10:HOME[0]*10+10,HOME[1]*10:HOME[0]*10+10,:] = (255,0,0) it is just to verify that we are indeed at right location
            draw_path()
            
            f, axes = plt.subplots(1, 2)
            sns.heatmap( data=optimal_value,  ax=axes[0])
            sns.heatmap(   data=optimal_path,  ax=axes[1])
            plt.show()
            #plt.figure(figsize=(8,10))
#             sns.heatmap(optimal_value)
#             plt.show()
#             #plt.figure(figsize=(8,10))
#             sns.heatmap(optimal_path)
#             plt.show()
        elif state == 3:
            #start game
            pass
        elif state == 4:
            # # New map img
            state=5
            # draw car
            # show color
            #terminators = [HOME]
            #print("terminating locations")
            #print(terminators)
            #no_actions = 4
            #Grid_shape = (50,50) #(100,100) # later set automatically
            img_mask = img.copy()   # img4.jpg # this is the fresh copy not affected by any thing 
            cv2.rectangle(img, (HOME[1]*width+width,HOME[0]*width-width),(HOME[1]*width-width,HOME[0]*width+width),(0,0,250),3)
            cv2.rectangle(img, (AGENT[1]*width+width,AGENT[0]*width-width),(AGENT[1]*width-width,AGENT[0]*width+width),(250,0,0),3)
            print()
            print("Calculating...")
            optimal_value,optimal_policy = Calculate_optimal(Grid_shape,no_actions, terminators,img_mask)
            print('OPTIMAL POLICY')
            print(optimal_value)
            l,m = optimal_value.shape  # (100,100)
            #print(img.shape)
            trace_path(optimal_value,optimal_policy,AGENT)
            #img[HOME[0]*10:HOME[0]*10+10,HOME[1]*10:HOME[0]*10+10,:] = (255,0,0) it is just to verify that we are indeed at right location
            draw_path()
            
            f, axes = plt.subplots(1, 2)
            sns.heatmap(  data=optimal_value, ax=axes[0])
            sns.heatmap(  data=optimal_path,   ax=axes[1])
            plt.show()
#             #plt.figure(figsize=(10,10))
#             sns.heatmap(optimal_value)
#             plt.show()
#             sns.heatmap(optimal_path)
#             plt.show()
            
            
    if event == cv2.EVENT_MOUSEMOVE:

        if drag ==True:
            state=4
            cv2.circle(img, (x,y),5,(0,0,0),15)
            #cv2.circle(img, (x,y),2,(0,0,0),4)
            

cv2.setMouseCallback("window", draw)

while True:
    cv2.imshow("window", img)
    key_val = cv2.waitKey(1)
    if key_val == ord("d"):
        # time to create new image
        optimal_path = np.zeros((50,50))
        img = cv2.imread("img4.jpg")[:size,:size,:]
    if key_val == ord("c"):
        state = 0
        optimal_path = np.zeros((50,50))
        img = cv2.imread("img4.jpg")[:size,:size,:]
    if key_val == 27:
        break
cv2.destroyAllWindows()
#cv2.imwrite("final_save.jpg",img)

In [ ]:
sns.heatmap(optimal_value)